# Undirected Graphs

### Prims algorithm / Dijkstra's Algorithm (O(ElogV)) [Using PQs]

Both are algorithmically same       
Prims algorithm for MST     
Dijkstra algorithm for SingleSourceShortest path   

O(V^2) without PQs, better for dense graphs

In [1]:
from heapq import *
g = {} # adjacency list of the graph

mstCost = 0
q = [(0 , 0)]
visited = set()
while len(q):
    weight , node = heappop(q)
    if node in visited: continue
    visited.add(node)
    mstCost += weight
    if len(visited)==len(g):
        break
    for v,w in g[node]:
        if v not in visited:
            heappush(q, (w, v))
print("MST Cost: ", mstCost)



### Kruskal's Algorithm (Disjoint Sets) (OElogE)

In [ ]:
class Solution:
    #Function to find sum of weights of edges of the Minimum Spanning Tree.
    def spanningTree(self, V, g):
         
        sets = [-1]*V
        def find(a):
            if sets[a]<0: return a
            sets[a]=find(sets[a])
            return sets[a]
        
        def union(a,b):
            A,B = find(a),find(b)
            if A==B: return False
            if sets[A] < sets[B]:
                sets[A] += sets[B]
                sets[B]=A
            else:
                sets[B] += sets[A]
                sets[A]=B
            return True

        edges = []
        for u in range(V):
            for v,w in g[u]:
                edges.append([u,v,w])
        edges.sort(key = lambda a:a[2])
        # print(edges)
        
        count = 0
        ans = 0
        for u,v,w in edges:
            if union(u,v):
                count += 1
                ans += w
            if count==V-1:
                break
                
        return ans


### Bellman Ford Algorithm  O(EV) == O(V^3)

* Suitable for negative edge graphs where Dijkstra doesn't work
* Should not have negative edge cycles

In [ ]:
# Bellman Ford Algorithm for Single Source shortest path
# Directed Graph with n vertices labeled from 1 to n-1
# list of edges (u,v,w)
def bellman(V, edges: list, src):
    d = [float('inf')]*V
    d[src]=0
    for i in range(V-1):
        for u,v,w in edges:
            if d[v] > d[u] + w:
                d[v]=d[u]
    return d

### Floyd Warshall Algorithm
* Should not have negative weight cycles

In [3]:
# this algorithm can be applied on adjacency matrix only
inf = float('inf')
g = [[inf,inf,1],[1,inf,inf], [inf,1,inf]]
n = len(g)
for k in range(n):
    for i in range(n):
        for j in range(n):
            g[i][j] = min(g[i][j], g[i][k] + g[k][j])
print(g)

[[3, 2, 1], [1, 3, 2], [2, 1, 3]]


### Find all Topological Sortings possible (Kahn's Algorithm)


In [9]:
g = {0:[], 1:[], 2:[3], 3:[1], 4:[0,1], 5:[0,2]}

def solve(g):
    V = len(g)
    indegree = [0]*V
    for u in range(V):
        for v in g[u]:
            indegree[v]+=1

    q = set()
    for i in range(V):
        if indegree[i]==0:
            q.add(i)

    temp = []
    res = []
    def recur(n=V):
        if n == 0:
            res.append(temp.copy())
            return
        nodes = list(q)
        for node in nodes:
            temp.append(node)
            q.remove(node)
            for v in g[node]:
                indegree[v]-=1
                if indegree[v]==0: q.add(v)
            recur(n-1)
            for v in g[node]:
                if indegree[v]==0: q.remove(v)
                indegree[v]+=1
            q.add(node)
            temp.pop()
    recur(V)
    return res
# ans = set([tuple(x) for x in solve(g)])

for x in sorted(solve(g)):
    print(x)
# print(solve(g))
        

[4, 5, 0, 2, 3, 1]
[4, 5, 2, 0, 3, 1]
[4, 5, 2, 3, 0, 1]
[4, 5, 2, 3, 1, 0]
[5, 2, 3, 4, 0, 1]
[5, 2, 3, 4, 1, 0]
[5, 2, 4, 0, 3, 1]
[5, 2, 4, 3, 0, 1]
[5, 2, 4, 3, 1, 0]
[5, 4, 0, 2, 3, 1]
[5, 4, 2, 0, 3, 1]
[5, 4, 2, 3, 0, 1]
[5, 4, 2, 3, 1, 0]


## Tarjan's Algorithm

### Print All Strongly Connected Components

In [7]:
class Solution:
    
    def tarjans(self, n, g):
        res = []
        time = [0]*n
        low  = [0]*n
        self.t = 1
        stack = []
        onStack = [False]*n
    
        def dfs(node):
            time[node] = low[node] = self.t
            self.t+=1
            stack.append(node)
            onStack[node]=True
            for child in g[node]:
                if time[child]==0:
                    dfs(child)
                    if onStack[child]:
                        low[node] = min(low[node], low[child])
                elif onStack[child]:
                    low[node] = min(low[node], time[child])
            if time[node] == low[node]:
                ans = []
                while True:
                    ans.append(stack.pop())
                    onStack[ans[-1]] = False
                    if ans[-1]==node: break
                res.append(ans)
            return
    
        for i in range(n):
            if time[i]==0:
                dfs(i)
        return res

visited = set()
n = 9
edges = [(2,8), (8,0), (7,5), (5,5)]
g = {i:[] for i in range(n)}
for u,v in edges:
    g[u].append(v)
print(Solution().tarjans(n,g))

[[0], [1], [8], [2], [3], [4], [5], [6], [7]]


### Articulation Points

In [7]:
# Algorithm to find Bridges is also very similar .. but a little easier
class Solution:
    def findAPs(self, g):
        n = len(g)
        self.t = 0
        time=[0]*n
        low = [0]*n
        ap = [False]*n # True if it is articulation point
        visited = set()
        
        def AP(node, parent=None):
            countChild = 0  # counts disconnected childrens
            low[node]=time[node]=self.t
            self.t+=1
            visited.add(node)
            for child in g[node]:
                if child==parent: continue
                if child not in visited:
                    countChild+=1
                    AP(child, node)
                    low[node] = min(low[node], low[child])
                    if parent!=None and low[child] >= time[node]:
                        ap[node]=True
                else:
                    low[node] = min(low[node], time[child])
            
            if parent==None and countChild>1:
                ap[node] = True
            return
        AP(0)
        return ap
          
                
g = [[1,2,3], [0,2], [0,1], [0,4], [3]] # given a connected graph
g1 = [[1], [0,2], [1,3], [2]]
print(Solution().findAPs(g))
print(Solution().findAPs(g1))
    

[True, False, False, True, False]
[False, True, True, False]


In [20]:
class Solution:
    
    def tarjans(self, n, g):
        res = []
        time = [0]*n
        low  = [0]*n
        self.t = 1
        visited = set()
    
        def dfs(node):
            time[node] = low[node] = self.t
            self.t+=1
            comp = [node]
            for child in g[node]:
                if time[child]==0:
                    childComp = dfs(child)
                    if time[node] < low[child]:
                        res.append(childComp)
                        [visited.add(x)  for x in childComp]
                    else:
                        comp.extend(childComp)
                if child not in visited:
                    low[node] = min(low[node], low[child])

            return comp
    
        for i in range(n):
            if time[i]==0:
                comp = dfs(i)
                res.append(comp)
                [visited.add(node) for node in comp]
        return res

# n = 9
# edges = [(2,8), (8,0), (7,5), (5,5)]
# n = 8
# edges = [
#     [4, 4],
# [3, 1],
# [0, 2],
# [6, 3],
# [6, 5],
# [1, 4],
# [1, 7],
# [3, 7],
# [1, 0],
# [3, 3],
# [4, 3],
# [1, 4],
# [7, 6],
# ]

n = 10
s = '''
5 4
6 6
3 9
9 5
2 1
3 0
9 9
1 3
4 5
8 5
7 3
3 8
9 5
5 3
8 6
'''.split()

# n = 8
# edges = [(0,1), (1,2), (2,0), (3,7), (7,3), (3,4), (7,5), (6,0), (5,0), (5,6), (6,4), (4,5)]
g = {i:[] for i in range(n)}
edges = set()
for i in range(0,len(s), 2):
    edges.add((int(s[i]), int(s[i+1])))
for u,v in edges:
    g[u].append(v)
for u in g:
    g[u]=list(g[u])
print(g)
print(Solution().tarjans(n,g))

{0: [], 1: [3], 2: [1], 3: [8, 0, 9], 4: [5], 5: [4, 3], 6: [6], 7: [3], 8: [6, 5], 9: [9, 5]}
[[0], [6], [3, 8, 5, 4, 9], [1], [2], [7]]
